In [ ]:
L = 100
N = 1000
M = 1000
dx = L/N
dk = 2π/L
dt = 1e-3

In [ ]:
x1 = dx .* collect(0:N-1)
x2 = dx .* collect(0:N-1)'
k1 = dk .* collect(0:N÷2)
k2 = dk .* collect(0:N÷2)';

In [ ]:
x' .* x

In [ ]:
const k = 2*π / L .* collect(1:(N÷2)+1)
const k² = k.*k